In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [4]:
dataset

User ID  Gender  Age  EstimatedSalary  Purchased
0    15624510    Male   19            19000          0
1    15810944    Male   35            20000          0
2    15668575  Female   26            43000          0
3    15603246  Female   27            57000          0
4    15804002    Male   19            76000          0
..        ...     ...  ...              ...        ...
395  15691863  Female   46            41000          1
396  15706071    Male   51            23000          1
397  15654296  Female   50            20000          1
398  15755018    Male   36            33000          0
399  15594041  Female   49            36000          1

[400 rows x 5 columns]

In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
dataset

User ID  Age  EstimatedSalary  Purchased  Gender_Male
0    15624510   19            19000          0            1
1    15810944   35            20000          0            1
2    15668575   26            43000          0            0
3    15603246   27            57000          0            0
4    15804002   19            76000          0            1
..        ...  ...              ...        ...          ...
395  15691863   46            41000          1            0
396  15706071   51            23000          1            1
397  15654296   50            20000          1            0
398  15755018   36            33000          0            1
399  15594041   49            36000          1            0

[400 rows x 5 columns]

In [7]:
dataset=dataset.drop("User ID" ,axis=1)

In [8]:
dataset

Age  EstimatedSalary  Purchased  Gender_Male
0     19            19000          0            1
1     35            20000          0            1
2     26            43000          0            0
3     27            57000          0            0
4     19            76000          0            1
..   ...              ...        ...          ...
395   46            41000          1            0
396   51            23000          1            1
397   50            20000          1            0
398   36            33000          0            1
399   49            36000          1            0

[400 rows x 4 columns]

In [9]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [10]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
from sklearn.tree import DecisionTreeClassifier

In [13]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']} 



grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 
 



Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [15]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)




In [16]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'sqrt', 'splitter': 'best'}: 0.88781000045891


In [17]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[78  7]
 [ 8 41]]


In [18]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.91      0.92      0.91        85
           1       0.85      0.84      0.85        49

    accuracy                           0.89       134
   macro avg       0.88      0.88      0.88       134
weighted avg       0.89      0.89      0.89       134



In [19]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.8771908763505402

In [20]:
table=pd.DataFrame.from_dict(re)

In [21]:
table

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.002393      0.001372         0.000000        0.000000   
1        0.002333      0.000596         0.000000        0.000000   
2        0.003110      0.001508         0.004913        0.003447   
3        0.002849      0.001016         0.006447        0.001561   
4        0.003544      0.000775         0.005733        0.001445   
5        0.002387      0.000693         0.005047        0.000983   
6        0.000403      0.000805         0.000000        0.000000   
7        0.001928      0.000574         0.000000        0.000000   
8        0.002827      0.000989         0.005123        0.000572   
9        0.003118      0.000812         0.004780        0.000748   
10       0.003078      0.001037         0.004438        0.000635   
11       0.002164      0.000182         0.004773        0.001651   

   param_criterion param_max_features param_splitter  \
0             gini               auto           best   
1             gini               auto         random   
2             gini               sqrt           best   
3             gini               sqrt         random   
4             gini               log2           best   
5             gini               log2         random   
6          entropy               auto           best   
7          entropy               auto         random   
8          entropy               sqrt           best   
9          entropy               sqrt         random   
10         entropy               log2           best   
11         entropy               log2         random   

                                               params  split0_test_score  \
0   {'criterion': 'gini', 'max_features': 'auto', ...                NaN   
1   {'criterion': 'gini', 'max_features': 'auto', ...                NaN   
2   {'criterion': 'gini', 'max_features': 'sqrt', ...           0.804764   
3   {'criterion': 'gini', 'max_features': 'sqrt', ...           0.808927   
4   {'criterion': 'gini', 'max_features': 'log2', ...           0.829615   
5   {'criterion': 'gini', 'max_features': 'log2', ...           0.782557   
6   {'criterion': 'entropy', 'max_features': 'auto...                NaN   
7   {'criterion': 'entropy', 'max_features': 'auto...                NaN   
8   {'criterion': 'entropy', 'max_features': 'sqrt...           0.829615   
9   {'criterion': 'entropy', 'max_features': 'sqrt...           0.869532   
10  {'criterion': 'entropy', 'max_features': 'log2...           0.826263   
11  {'criterion': 'entropy', 'max_features': 'log2...           0.822092   

    split1_test_score  split2_test_score  split3_test_score  \
0                 NaN                NaN                NaN   
1                 NaN                NaN                NaN   
2            0.868752           0.832483           0.906166   
3            0.811321           0.722205           0.814409   
4            0.868752           0.796284           0.869709   
5            0.828959           0.795256           0.868632   
6                 NaN                NaN                NaN   
7                 NaN                NaN                NaN   
8            0.811321           0.813179           0.851527   
9            0.849057           0.775815           0.849057   
10           0.808392           0.832483           0.832483   
11           0.831253           0.720804           0.869709   

    split4_test_score  mean_test_score  std_test_score  rank_test_score  
0                 NaN              NaN             NaN                9  
1                 NaN              NaN             NaN                9  
2            0.885265         0.859486        0.036478                1  
3            0.867097         0.804792        0.046593                8  
4            0.905069         0.853886        0.037412                2  
5            0.862442         0.827569        0.034567                6  
6                 NaN              NaN             NaN         

In [22]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Gender Male 0 or 1:"))

Age: 28
Salary: 20000
Gender Male 0 or 1: 1


In [23]:
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
